In [47]:
import pandas as pd
import os
from tqdm import tqdm
import urllib.request
import json
import requests
import time
import csv



In [21]:
def similaritysort(e):
  return e[1]

In [44]:
# augment gene set library: 
def coexpression_augment(file_link):
    augmented_df_data = []
    GENESHOT_URL = 'https://maayanlab.cloud/geneshot/api/associate'
    with urllib.request.urlopen(file_link) as f:
        html = f.read().decode('utf-8') 
        for line in tqdm(html.split('\n'), total=len(html.split('\n'))):
            line_content = line.split('\t')
            geneset_name =  line_content[0]
            genes = line_content[1:]
            genes = [i for i in genes if i != '']
            payload = {
                "gene_list": genes,
                "similarity": "coexpression" 
                }
            try:
                response = requests.post(GENESHOT_URL, json=payload)
                if response.ok:
                    data = json.loads(response.text)
                    associationData = data['association']
                    gene_similarity = []
                    for gene in list(associationData.keys()):
                        gene_similarity.append([gene, associationData[gene]['simScore']])
                    gene_similarity.sort(key=similaritysort)
                    top_genes = []
                    for pair in gene_similarity[:100]: 
                        top_genes.append(pair[0])
                    augmented_df_data.append([geneset_name, top_genes])   
            except:
                time.sleep(10)
                data = json.loads(response.text)
                associationData = data['association']
                gene_similarity = []
                for gene in list(associationData.keys()):
                    gene_similarity.append([gene, associationData[gene]['simScore']])
                gene_similarity.sort(key=similaritysort)
                top_genes = []
                for pair in gene_similarity[:100]: 
                    top_genes.append(pair[0])
                augmented_df_data.append([geneset_name, top_genes])  
        augmented_df = pd.DataFrame(augmented_df_data, columns=['Term', 'Genes'])
        return augmented_df

In [45]:
co_expressed_azimuth = coexpression_augment('https://minio.dev.maayanlab.cloud/g2sg/CFDE%20libraries/Hubmap_Azimuth_2023.txt')

100%|██████████| 1426/1426 [33:29<00:00,  1.41s/it]


In [73]:
# read to .gmt file
with open("HubMAP_Azimuth_2023_Augmented.gmt", "w") as o:
    csv_writer = csv.writer(o, delimiter="\t")
    df = co_expressed_azimuth
    terms = df["Term"].values
    for term in terms:
        genes = df[df['Term'] == term]['Genes'].values[0]
        csv_writer.writerow([
            term,
            "",
            *list(genes)
        ])